# Dynamic Programming

### Question 32: Longest Valid Parenthesis
Given a string containing just the characters '(' and ')', find the length of the longest valid (well-formed) parentheses substring

In [ ]:
"""
Scan from left to right, if right > left: set count = 0
Scan from right to left, if left > right: set count = 0
"""
def longestValidParentheses(s):
    if len(s) == 0:
        return 0
    # Left traverse
    left_count = [0]*len(s)
    left_left = 0
    left_right = 0
    for i in range(len(s)):
        if s[i] == "(":
            left_left += 1
        elif s[i] == ")":
            left_right += 1
        if left_left == left_right:
            left_count[i] = left_count[i-2*left_left] + 2*left_left
            left_left = left_right = 0
        if left_right > left_left:
            left_left = left_right = 0
            left_count[i] = 0
    # Right traverse
    right_count = [0]*len(s)
    right_left = 0
    right_right = 0
    for i in range(len(s)):
        if s[-(i+1)] == "(":
            right_left += 1
        elif s[-(i+1)] == ")":
            right_right += 1
        if right_left == right_right:
            right_count[-(i+1)] = right_count[-(i+1)+2*right_left] + 2*right_left
            right_left = right_right = 0
        if right_left > right_right:
            right_left = right_right = 0
            right_count[-(i+1)] = 0
    return max(max(right_count), max(left_count))

### Question 44: Wildcard Matching
Given an input string (s) and a pattern (p), implement wildcard pattern matching with support for '?' and '*' where:
'?' Matches any single character.
'*' Matches any sequence of characters (including the empty sequence).
The matching should cover the entire input string (not partial)

In [62]:
"""
First we create an m*n matrix
    -> If s[i] == p[j]: then corresponding boolean equals [i-1][j-1]th index
    -> If p[j] == "?", then corresponding is true if [i-1][j-1] is true
    -> If p[j] == "*", then if either [i-1][j] or [i][j-1] is true: corresponding is true

Only at the very beginning of the string p do we have to consider cases which we
change the first line of the matrix. Otherwise if s[i] == p[j], M[i+1][j+1] = M[i][j]
Consider:
  a * a d
a
b
c
d
Then the previous attempt "if p[j-1] == "*": M[i+1][j+1] = M[i+1][j] or M[i][j]" would be invalid
"""
import numpy as np
def isMatch(s, p):
    # Initialize the matrix. The first row, first column are zeros besides the topleft index
    M = np.zeros((len(s)+1,len(p)+1)).astype(int)
    M[0][0] = 1
    for i in range(len(p)):
        if p[i] == "*":
            M[0][i+1] = M[0][i]
    for i in range(len(s)):
        for j in range(len(p)):
            if s[i] == p[j]:
                #if p[j-1] == "*":
                 #   M[i+1][j+1] = M[i+1][j] or M[i][j]
                #else:
                M[i+1][j+1] = M[i][j]
            elif p[j] == "?":
                M[i+1][j+1] = M[i][j]
            elif p[j] == "*":
                M[i+1][j+1] = M[i][j+1] or M[i+1][j] or M[i][j]
            else:
                M[i+1][j+1] = 0
    return bool(M[-1][-1])

### Question 221: Maximal Square
Given an m x n binary matrix filled with 0's and 1's, find the largest square containing only 1's and return its area.

In [117]:
"""
Use backtracking: start from the bottom right corner
When a cell's all three incoming directions are 1: add 1
If any of them is zero: its value equals itself
""" 
import numpy as np
def maximalSquare(matrix):
    # First we need to fill out the base cases
    R,C = len(matrix),len(matrix[0])
    mat = np.zeros((R,C)).astype(int)
    # Case 1: When there are more rows than columns
    if R > C:
        for i in range(R-1,C-1,-1):
            for j in range(len(matrix[0])-1,-1,-1):
                if i == R-1 or j == C-1:
                    mat[i][j]=matrix[i][j]
                else:
                    down = mat[i+1,j]
                    right = mat[i,j+1]
                    diag = mat[i+1,j+1]
                    
                    mat[i][j] = int(matrix[i][j])*(1+min(down,right,diag))
    # Case 2: When there are more columns than rows
    if C > R:
        for i in range(R-1,-1,-1):
            for j in range(C-1,R-1,-1):
                if i == R-1 or j == C-1:
                    mat[i][j]=matrix[i][j]
                else:
                    down = mat[i+1,j]
                    right = mat[i,j+1]
                    diag = mat[i+1,j+1]
                    
                    mat[i][j] = int(matrix[i][j])*(1+min(down,right,diag))
    
    # Base case for the remaining square
    L = min(R,C)
    # Case 1: if number of rows > number of columns
    if len(matrix) > L:
        mat[L-1][-1] = matrix[L-1][-1]
        for i in range(len(matrix[0])-2,-1,-1):
            mat[i][-1] = matrix[i][-1]
            down = mat[L,i]
            right = mat[L-1,i+1]
            diag = mat[L,i+1]
            mat[L-1][i] = int(matrix[L-1][i])*(1+min(down,right,diag))
    # Case 2: if number of columns > column of rows
    if len(matrix[0]) > L:
        mat[L-1] = matrix[L-1]
        for i in range(L-2,-1,-1):
            down = mat[i+1,L-1]
            right = mat[i,L]
            diag = mat[i+1,L]
            mat[i][L-1] = int(matrix[i][L-1])*(1+min(down,right,diag))
    # Case 3: if number of rows = number of columns
    if len(matrix) == len(matrix[0]) == L:
        mat[-1] = matrix[-1]
        for i in range(L):
            mat[i][-1] = matrix[i][-1]
    
    # Then use dynamic programming to solve the remaining values
    for i in range(L-2,-1,-1):
        for j in range(i,-1,-1):
            mat[i,j] = int(matrix[i][j])*(1+min(mat[i+1,j],mat[i,j+1],mat[i+1,j+1]))
            mat[j,i] = int(matrix[j][i])*(1+min(mat[j+1,i],mat[j,i+1],mat[j+1,i+1]))
    # Find the max in the matrix
    M = 0
    for i in mat:
        M = max(M,max(i))
    return M**2